In [1]:
#installing necessary packages
import sys
stdout = sys.stdout

sys.stdout = stdout
!{sys.executable} -m pip install git+https://github.com/huggingface/transformers
!{sys.executable} -m pip install bert-serving-server bert-serving-client
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

  Cloning https://github.com/huggingface/transformers to c:\users\joshg\appdata\local\temp\pip-req-build-apcc8g_7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for transformers: filename=transformers-4.14.0.dev0-py3-none-any.whl size=3320943 sha256=51ec39a71a982e0ab1d8ab65a713747fc65bfc2ed4d98ac5006c08083d16bb86
  Stored in directory: C:\Users\joshg\AppData\Local\Temp\pip-ephem-wheel-cache-bg90doj5\wheels\14\a0\7b\8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0
    Uninstalling transformers-4.13.0:
      Successfully uninstalled transformers-4.13.0


  Running command git clone -q https://github.com/huggingface/transformers 'C:\Users\joshg\AppData\Local\Temp\pip-req-build-apcc8g_7'
You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\joshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
import numpy as np
import pandas as pd
import re

data = pd.read_csv('test.csv')

# execute chosen function and print results
def getResults(questions, fn):
    def getResult(q):
        [answer, score, prediction] = fn(q)
        return [q, prediction, answer, score]

    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

# test set of questions
test_data = [
    "What is the population of Egypt?",
    "What is the poplation of egypt",
    "How long is a leopard's tail?",
    "Do you know the length of a leopard's tail?",
    "biggest city in Finland"
]
    


In [10]:
def matchQuestion(q):
    # determine whether question matches at least part of a question in the dataset
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Could not find an answer", 0, ""


In [12]:
getResults(test_data, matchQuestion)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,78 million,1
1,What is the poplation of egypt,,"Sorry, I didn't get you.",0
2,How long is a leopard's tail?,How long is a leopard's tail?,60-110cm,1
3,Do you know the length of a leopard's tail?,,"Sorry, I didn't get you.",0
4,biggest city in Finland,What is the biggest city in Finland?,Greater Helsinki,1


In [13]:
def lratio(s, t):
    
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # Else cost is 1; calculate ratio
                cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions

    ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
    return ratio

In [14]:
def approxAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    
    for idx, row in data.iterrows():
        score = lratio(row["Question"], q) # calculate levenshtein ratio to compare question strings
        if score >= 0.9:  # Immediately return row for high scores 
            return  row["Answer"], score, row["Question"]
        elif score > max_score: # Otherwise, continue comparing
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]

    if max_score > 0.8: # Adjustable score threshold
        return answer, max_score, prediction
    return "Could not find an answer", max_score, prediction

getResults(test_data, approxAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,78 million,1.000000
1,What is the poplation of egypt,What is the population of Egypt?,78 million,0.951613
2,How long is a leopard's tail?,How long is a leopard's tail?,60-110cm,1.000000
3,Do you know the length of a leopard's tail?,How long is a leopard's tail?,"Sorry, I didn't get you.",0.750000
4,biggest city in Finland,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.779661


In [15]:
def improvedApproxAnswer(q): #combines the naive string matching method and the levenshtein ratio calculation
    # if question matches all or part of the original question, score = 1
    max_score = 0
    answer = ""
    prediction = ""
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    for idx, row in data.iterrows():
        score = lratio(row["Question"], q)
        if score >= 0.9:  # Immediately return row for high scores 
            return  row["Answer"], score, row["Question"]
        elif score > max_score: # Otherwise, continue comparing
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]

    if max_score > 0.8: # Adjustable score threshold
        return answer, max_score, prediction
    return "Could not find an answer", max_score, prediction

getResults(test_data, improvedApproxAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,78 million,1.000000
1,What is the poplation of egypt,What is the population of Egypt?,78 million,0.951613
2,How long is a leopard's tail?,How long is a leopard's tail?,60-110cm,1.000000
3,Do you know the length of a leopard's tail?,How long is a leopard's tail?,"Sorry, I didn't get you.",0.750000
4,biggest city in Finland,What is the biggest city in Finland?,Greater Helsinki,1.000000


In [16]:
data = pd.read_csv('JeopardyTest.csv')
test_data = [
    "Galileo was under house arrest for espousing this man's theory",
    "Why was Galileo under house arrest?",
    "What insect shares billing with a grasshopper",
    "Who was the second President of the United States",
    "What do cows regurgitate from their stomach and chew?",
    "What Indian Ocean Island is known for its tea?",
    "Who was No.8 for the Barons and the Bulls?",
    "This outlaw was 'murdered by a traitor and a coward whose name is not worthy to appear here'",
    "What index rates the intensity of the sun's radiation"
]

getResults(test_data, improvedApproxAnswer)

,Q,Prediction,A,Score
0,Galileo was under house arrest for espousing t...,"For the last 8 years of his life, Galileo was ...",Copernicus,1.000000
1,Why was Galileo under house arrest?,"For the last 8 years of his life, Galileo was ...","Sorry, I didn't get you.",0.503817
2,What insect shares billing with a grasshopper,"In the title of an Aesop fable, this insect sh...","Sorry, I didn't get you.",0.721311
3,Who was the second President of the United States,"Signer of the Dec. of Indep., framer of the Co...","Sorry, I didn't get you.",0.620915
4,What do cows regurgitate from their stomach an...,Cows regurgitate this from the first stomach t...,"Sorry, I didn't get you.",0.674603
5,What Indian Ocean Island is known for its tea?,In 1000 Rajaraja I of the Cholas battled to ta...,"Sorry, I didn't get you.",0.609929
6,Who was No.8 for the Barons and the Bulls?,"No. 8: 30 steals for the Birmingham Barons; 2,...","Sorry, I didn't get you.",0.625000
7,This outlaw was 'murdered by a traitor and a c...,"Outlaw: ""Murdered by a traitor and a coward wh...",Jesse James,0.926136
8,What index rates the intensity of the sun's ra...,"On June 28, 1994 the nat'l weather service beg...","Sorry, I didn't get you.",0.625000


In [21]:
def improvedApproxAnswer2(q): #adjusted score values for comparison with previous improvedApproxAnswer function
    # if question matches all or part of the original question, score = 1
    max_score = 0
    answer = ""
    prediction = ""
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    for idx, row in data.iterrows():
        score = lratio(row["Question"], q)
        if score >= 0.9:  # Immediately return row for high scores 
            return  row["Answer"], score, row["Question"]
        elif score > max_score: # Otherwise, continue comparing
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]

    if max_score > 0.62: # Adjustable score threshold
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction

getResults(test_data, improvedApproxAnswer2)

,Q,Prediction,A,Score
0,Galileo was under house arrest for espousing t...,"For the last 8 years of his life, Galileo was ...",Copernicus,1.000000
1,Why was Galileo under house arrest?,"For the last 8 years of his life, Galileo was ...","Sorry, I didn't get you.",0.503817
2,What insect shares billing with a grasshopper,"In the title of an Aesop fable, this insect sh...",the ant,0.721311
3,Who was the second President of the United States,"Signer of the Dec. of Indep., framer of the Co...",John Adams,0.620915
4,What do cows regurgitate from their stomach an...,Cows regurgitate this from the first stomach t...,the cud,0.674603
5,What Indian Ocean Island is known for its tea?,In 1000 Rajaraja I of the Cholas battled to ta...,"Sorry, I didn't get you.",0.609929
6,Who was No.8 for the Barons and the Bulls?,"No. 8: 30 steals for the Birmingham Barons; 2,...",Michael Jordan,0.625000
7,This outlaw was 'murdered by a traitor and a c...,"Outlaw: ""Murdered by a traitor and a coward wh...",Jesse James,0.926136
8,What index rates the intensity of the sun's ra...,"On June 28, 1994 the nat'l weather service beg...",the UV index,0.625000
